In [2]:
import pandas as pd

df = pd.read_excel('SIB-200 languages - ACL.xlsx')
filtered_languages = df[df['Bloom Train Data Percentage'] > 0]
filtered_languages

,Folder Name,Language Name,ISO 639 - 1,ISO 639 - 2 (B),ISO 639 - 2 (T),ISO 639 - 3,Script (ISO 15924),Language Family,Population,Language Vitality,Digital Language Support,Resource Level,Bloom Train Data Percentage,XLM-R Train Tokens
7,aka_Latn,Akan,ak,aka,aka,aka,Latn,Niger-Congo,1 million to 1 billion,Institutional,Ascending,1,0.00007,NaN
11,arb_Arab,Standard Arabic,NaN,NaN,NaN,arb,Arab,Afro-Asiatic,1 million to 1 billion,Institutional,Thriving,5,4.60000,2869.0
16,asm_Beng,Assamese,as,asm,asm,asm,Beng,Indo-European,1 million to 1 billion,Institutional,Vital,1,0.01000,5.0
23,bam_Latn,Bamanankan,bm,bam,bam,bam,Latn,Niger-Congo,1 million to 1 billion,Institutional,Ascending,1,0.00004,NaN
27,ben_Beng,Bengali,bn,ben,ben,ben,Beng,Indo-European,1 million to 1 billion,Institutional,Vital,3,0.50000,525.0
35,cat_Latn,Catalan,ca,cat,cat,cat,Latn,Indo-European,1 million to 1 billion,Institutional,Thriving,4,1.10000,1752.0
48,eng_Latn,English,en,eng,eng,eng,Latn,Indo-European,1 billion plus,Institutional,Thriving,5,30.04000,55608.0
51,eus_Latn,Euskera,eu,baq,eus,eus,Latn,Isolate,1 million to 1 billion,Institutional,Vital,4,0.15000,270.0
56,fon_Latn,Fon,NaN,fon,fon,fon,Latn,Niger-Congo,1 million to 1 billion,Institutional,Emerging,0,0.00020,NaN
57,fra_Latn,French,fr,fre,fra,fra,Latn,Indo-European,1 million to 1 billion,Institutional,Thriving,5,12.90000,9780.0


In [3]:
import os
import subprocess
import torch
import gc

# Set environment variables
output_directory = "output_sib200"
os.makedirs(output_directory, exist_ok=True)
MAX_LENGTH = "164"
BATCH_SIZE = "16"
NUM_EPOCHS = "10"
SAVE_STEPS = "500000"
MODEL = "bigscience/bloom-1b1"
MODEL_TYPE = "bloom"
dir_name = "sib-200/data/annotated"
SEED = "42"

# Iterate over directories in dir_name
for SRC_LANG_DIR in filtered_languages['Folder Name'][:1]:
    gc.collect()
    torch.cuda.empty_cache()
    # Extract SRC_LANG from directory name
    SRC_LANG = os.path.basename(SRC_LANG_DIR)
    print(SRC_LANG)

    OUTPUT_FILE = f"test_result_{SRC_LANG}"
    OUTPUT_PREDICTION = f"test_predictions_{SRC_LANG}"
    SRC_DATA_DIR = os.path.join(dir_name, SRC_LANG)
    OUTPUT_DIR = f"{output_directory}/{MODEL}/{SRC_LANG}_bert"

    subprocess.run([
        'python', 'sib-200/code/train_textclass.py',
        "--data_dir", SRC_DATA_DIR,
        "--model_type", MODEL_TYPE,
        "--model_name_or_path", MODEL,
        "--output_dir", OUTPUT_DIR,
        "--output_result", OUTPUT_FILE,
        "--output_prediction_file", OUTPUT_PREDICTION,
        "--max_seq_length", MAX_LENGTH,
        "--num_train_epochs", NUM_EPOCHS,
        "--learning_rate", "1e-5",
        "--per_gpu_train_batch_size", BATCH_SIZE,
        "--per_gpu_eval_batch_size", BATCH_SIZE,
        "--save_steps", SAVE_STEPS,
        "--seed", SEED,
        "--gradient_accumulation_steps", "2",
        "--labels", os.path.join(SRC_DATA_DIR, 'labels.txt'),
        "--do_train",
        "--do_eval",
        "--do_predict",
        "--overwrite_output_dir"
    ])

    # Remove unnecessary files
    files_to_remove = [
        "pytorch_model.bin",
        "sentencepiece.bpe.model",
        "tokenizer.json",
        "tokenizer_config.json",
        "config.json",
        "training_args.bin",
        "special_tokens_map.json",
        "sentencepiece.model",
    ]

    for file_name in files_to_remove:
        file_path = os.path.join(OUTPUT_DIR, file_name)
        if os.path.exists(file_path):
            os.remove(file_path)

aka_Latn


01/04/2024 19:48:56 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False,
Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-1b1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
01/04/2024 19:49:03 - INFO - __main__ -   Training/evaluation parameters Namespace(data_dir='sib-200/data/annotated/aka_Latn', model_type='bloom', model_name_or_path='bigscience/bloom-1b1', output_dir='output_sib200/bigscience/bloom-1b1/aka_Latn_bert', output_prediction_file='test_predictions_aka_Latn', output_result='test_result_aka_Latn', labels='sib-200/data/annotated/aka_Latn/labels.txt', config_name='', tokenizer_name='', cache_dir='', max_seq_length=164, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, do_lower_case=False, per_gpu_train_batch_size=16, per_gpu_eval_batch